<a href="https://colab.research.google.com/github/s36934512/tensorpack/blob/master/GAN_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from PIL import Image
from tensorflow import keras
from tensorflow.keras import datasets,layers,optimizers,Sequential,metrics

TensorFlow 2.x selected.


In [ ]:
class Generator(keras.Model):
  def __init__(self):
    super(Generator,self).__init__()
    #encoder
    self.fc_layer_1 = layers.Dense(3*3*512)
    self.conv_1 = layers.Conv2DTranspose(256,3,3,'valid')

    self.bn_1 = layers.BatchNormalization()       
    self.conv_2 = layers.Conv2DTranspose(128,5,2,'valid')
    self.bn_2 = layers.BatchNormalization()     
    self.conv_3 = layers.Conv2DTranspose(3,4,3,'valid')

  def call(self, inputs, training=None):
    x = self.fc_layer_1(inputs)
    x = tf.reshape(x,[-1,3,3,512])
    x = tf.nn.relu(x)
    x = self.bn_1(self.conv_1(x),training=training)
    x = self.bn_2(self.conv_2(x),training=training)
    x = self.conv_3(x)
    x = tf.tanh(x)
    return x

In [ ]:
class Discriminator(keras.Model):
  def __init__(self):
    super(Discriminator,self).__init__()

    self.conv_1 = layers.Conv2D(64,5,3,'valid')
    self.conv_2 = layers.Conv2D(128,5,3,'valid')
    self.bn_1 = layers.BatchNormalization()
    self.conv_3 = layers.Conv2D(256,5,3,'valid')    
    self.bn_2 = layers.BatchNormalization()
    self.flatten = layers.Flatten()
    self.fc_layer = layers.Dense(1)

  
  def call(self, inputs, training=None):
    x = tf.nn.leaky_relu(self.conv_1(inputs))    
    x = tf.nn.leaky_relu(self.bn_1(self.conv_2(x),training=training))    
    x = tf.nn.leaky_relu(self.bn_2(self.conv_3(x),training=training))  
    x = self.flatten(x)
    x = self.fc_layer(x)
    return x

In [ ]:
g = Generator()
d = Discriminator()

In [ ]:
x = tf.random.normal([1,64,64,3])
z = tf.random.normal([1,100])
prob = g(x)
print(prob)
out = d(x)
print(out.shape)

tf.Tensor(
[[[[-5.07458695e-04  8.00373848e-04 -2.39310582e-04]
   [-1.42937497e-04  1.85758283e-04 -1.39470329e-03]
   [-1.79153786e-03 -2.83525093e-04 -4.73423075e-04]
   ...
   [ 1.15402823e-03  1.08104423e-05 -6.01928565e-04]
   [-3.33229073e-05  6.26340916e-04 -7.83988391e-04]
   [ 2.69759941e-04  4.90645878e-04 -1.81727938e-03]]

  [[-5.21908281e-04 -3.95908923e-04  3.45010514e-04]
   [-1.45137732e-04  6.46646658e-04  8.26917938e-04]
   [ 5.04842727e-04 -2.88895426e-05  1.63484452e-04]
   ...
   [-2.82559398e-04 -2.52541358e-05  8.27054086e-04]
   [-1.22206495e-03  3.18750564e-04 -5.44366194e-04]
   [-2.49203295e-04 -3.21527012e-04  8.03112707e-05]]

  [[-6.25745975e-04  4.23897363e-05  7.14368362e-04]
   [-4.04407911e-04 -2.18596979e-04  7.80994160e-05]
   [ 8.25141964e-04  8.28696648e-04  7.44889840e-04]
   ...
   [-1.55589706e-03 -5.25643001e-04 -2.30434904e-04]
   [ 4.77236958e-04  1.80016519e-04 -3.04282730e-04]
   [ 6.64343883e-04  1.79379457e-03 -6.86786429e-04]]

  ...

 